<a href="https://colab.research.google.com/github/kavyajeetbora/monitoring_water_surface_area/blob/master/notebooks/estimating_lake_depth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Monitoring Inland Surface Water Area

In [1]:
!pip install -q rioxarray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 820.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 44.8 MB/s eta 0:00:00


In [60]:
import geemap
import ee
import rioxarray as rxr
import scipy as sp
import numpy as np
import plotly.graph_objects as go
import matplotlib

ee.Authenticate()
ee.Initialize(project='kavyajeetbora-ee')

## Area of Interest

Used this tool to create a geojson file of the area of interest:

[keene Polyline Tool](https://www.keene.edu/campus/maps/tool/?coordinates=77.1200409%2C%2011.5324541%0A76.9923248%2C%2011.5062217%0A76.9916382%2C%2011.3467571%0A77.1529998%2C%2011.4261642%0A77.1200409%2C%2011.5324541) : Use this tool to create a polygon and generate a GeoJson text for further use

In [3]:
geojson  = {
  "coordinates": [
    [
      [
        77.1200409,
        11.5324541
      ],
      [
        76.9941101,
        11.5006165
      ],
      [
        76.9701462,
        11.4243672
      ],
      [
        77.0055084,
        11.3198347
      ],
      [
        77.1529998,
        11.4261642
      ],
      [
        77.1200409,
        11.5324541
      ]
    ]
  ],
  "type": "Polygon"
}

geometry = ee.Geometry(geojson)

## Download and Visualize the Lake Bathymetric Data

Estimating the lake volume using the GLOBathy - Global bathymetric survey data for lakes

[Global lakes bathymetry dataset](https://gee-community-catalog.org/projects/globathy/): bathymetric data of 1.4+ million waterbodies to align with the well-established global dataset, HydroLAKES. GLOBathy uses a GIS-based framework to generate bathymetric maps based on the waterbody maximum depth estimates and HydroLAKES geometric/geophysical attributes of the waterbodies. The maximum depth estimates are validated at 1,503 waterbodies, making use of several observed data sources

In [77]:
globathy = ee.Image("projects/sat-io/open-datasets/GLOBathy/GLOBathy_bathymetry")

In [5]:
globathy = ee.Image("projects/sat-io/open-datasets/GLOBathy/GLOBathy_bathymetry")
globathy = globathy.rename('Depth_m').unmask(0) ## Multiplying -1 to represent the data in negative

## Visualize the global bathymetry data
Map = geemap.Map()
visParams = {"min": 0, "max": 20, 'palette': ['#eff3ff','#c6dbef','#9ecae1','#6baed6','#3182bd','#08519c']}
Map.addLayer(globathy.clip(geometry), visParams, 'Global Bathymetry') ## Clip the image
Map.addLayer(geometry) ## To show the bounding area of interest geometry
Map.centerObject(geometry, zoom=12)
Map

Map(center=[11.437008883357802, 77.05579366889911], controls=(WidgetControl(options=['position', 'transparent_…

### Calculate the Total Volume

Here to calculate the lake volume, I have simply multiplied the pixel area with pixel depth and calculated the total sum.

Total Volume $ = \sum_{i=1}^{n} A_i * D_i$

There could be some better way to estimate the volume from bathymetry survey but for simplicity, I have used this

In [6]:
## mask the values where depths are 0
area_mask = globathy.gt(0)

## Calculate the area of each pixel in m2, rename the band as area_m2
area = area_mask.multiply(ee.Image.pixelArea()).rename('area_m2')

## Now calculate the volume of each pixel in m3, rename the band as volume_m3
volume = area.multiply(globathy).rename("volume_m3")

Calculate the total volume

In [89]:
volume = volume.select("volume_m3")
totalVolume = volume.reduceRegion(
    reducer = ee.Reducer.sum(),
    geometry = geometry,
    scale=30,
    maxPixels = 1e10
)

totalVolume = ee.Number(totalVolume.get('volume_m3')).divide(1e3).round()
## Total volume in ML
totalVolume

## Export the image as tif file

In [90]:
spatial_resolution = 30
globath_depths = globathy.multiply(-1)
geemap.ee_export_image(
    globath_depths, filename="lake_terrain.tif", scale=spatial_resolution, region=geometry, file_per_band=False
)

Generating URL ...
Please wait ...
Data downloaded to /content/lake_terrain.tif


### Visualizing the lake in 3D

In [93]:
def matplotlib_to_plotly(cmap, pl_entries):
    '''Converts a matplolib colorscale to plotly colorscale'''
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

def smoothen_dataArray(tif_file, sigma = 5):
    '''
    This function reads an tif image containing the terrain data
    Smoothens the elevation data using gaussian filter based on sigma value
    Input: .tif file
    returns: a processed Xarray.Dataset containing the terrain data
    '''
    da = rxr.open_rasterio(filename=tif_file)
    da_vals = da.isel(band=0).values ## Select the first band of the image
    # Apply gaussian filter, with sigmas as variables. Higher sigma = more smoothing and more calculations. Downside: min and max values do change due to smoothing
    sigma = [sigma, sigma] ## Sigma values in x and y direction
    z_smoothed = sp.ndimage.gaussian_filter(da_vals, sigma)

    da.data = np.expand_dims(z_smoothed, axis=0)
    return da


def plot_3D_terrain(xr_array, total_vol, colorscale = 'Earth', spatial_resolution=30, depth_scale=1):

    '''
    Plots an array data in 3D.
    Input: Xarray.DataArray
    Returns: plotly figure representing the 3D terrain data
    '''

    Z = xr_array.values ## Z are the elevation/depth values
    Y = xr_array['y'].values ## Y - latitude values
    X = xr_array['x'].values ## X - longitude values
    x_ratio, y_ratio = xr_array.shape

    fig = go.Figure()

    fig.add_trace(
        go.Surface(
            z=Z,
            x=X,
            y=Y,
            colorscale = colorscale,
            hovertemplate ='<b>Depth</b>: %{z:.2f} m',
            name=""
        )
    )

    fig.update_layout(
        margin=dict(l=0, r=0, t=30, b=0),
        title = f"Lake Topography | Total Estimated Volume: {total_vol} ML",
        plot_bgcolor="rgba(0, 0, 0, 0)",   # Transparent plot background
    )

    # Set the box aspect ratio (equal scales for all axes)
    fig.update_scenes(
        aspectratio=dict(x=spatial_resolution, y=spatial_resolution,z=depth_scale),
        xaxis=dict(showticklabels=False, title="", showgrid=False),
        yaxis=dict(showticklabels=False, title="", showgrid=False),
        zaxis=dict(showticklabels=False, title="Depth (m)<br>→", showgrid=False)
    )

    return fig

In [94]:
## Convert colorscales from matplotlib to plotly colorscales
terrain_cmap = matplotlib.cm.get_cmap('winter')
terrain = matplotlib_to_plotly(terrain_cmap, 255)

## Read and smoothen the DEM tif image file
ds = smoothen_dataArray('lake_terrain.tif', sigma=5)
da = ds.isel(band=0) ## Select the first array

## Plot the 3D terrain
total_vol = totalVolume.getInfo()
fig = plot_3D_terrain(da, total_vol, colorscale=terrain)
fig.show()

## Export it to html

In [72]:
fig.write_html('lake_terrain.html')

## References

1. Khazaei, Bahram; Read, Laura K; Casali, Matthew; Sampson, Kevin M; Yates, David N (2022): GLOBathy Bathymetry Rasters. figshare.
Dataset. https://doi.org/10.6084/m9.figshare.13404635.v1
2. [colorscales in plotly](https://plotly.com/python/builtin-colorscales/) and [colorscales in matplotlib](https://matplotlib.org/stable/users/explain/colors/colormaps.html)
3. [Constructing custom colorscale on plotly](https://plotly.com/python/colorscales/#constructing-a-discrete-or-discontinuous-color-scale)
4. [3D Terrain in Python by Jack McKew](https://jackmckew.dev/3d-terrain-in-python.html)
5. [Layout scenes in plotly](https://plotly.com/python/reference/layout/scene/)